# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,10.25,50,75,10.29,CL,1699466568
1,1,papatowai,-46.5619,169.4708,8.70,88,100,3.62,NZ,1699466612
2,2,along,28.1667,94.7667,14.38,75,13,0.88,IN,1699466612
3,3,rawson,-43.3002,-65.1023,14.80,50,43,7.82,AR,1699466354
4,4,port mathurin,-19.6833,63.4167,24.99,77,100,5.21,MU,1699466613


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_data = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 22) \
    & (city_data_df["Wind Speed"] < 3) & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
narrowed_data = narrowed_data.dropna()

# Display sample data
narrowed_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
196,196,arar,30.9753,41.0381,22.01,30,0,2.57,SA,1699466673
295,295,bushehr,28.9684,50.8385,25.05,61,0,2.06,IR,1699466694
375,375,south hedland,-20.4089,118.5983,25.96,83,0,2.57,AU,1699466716
418,418,cullowhee,35.3137,-83.1765,26.09,52,0,0.00,US,1699466736
440,440,tamanrasset,22.7850,5.5228,23.95,9,0,2.06,DZ,1699466744
534,534,pasni,25.2631,63.4710,26.27,69,0,2.60,PK,1699466766
572,572,la toma,-33.0526,-65.6238,25.25,15,0,2.34,AR,1699466778


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_data[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
196,arar,SA,30.9753,41.0381,30,
295,bushehr,IR,28.9684,50.8385,61,
375,south hedland,AU,-20.4089,118.5983,83,
418,cullowhee,US,35.3137,-83.1765,52,
440,tamanrasset,DZ,22.7850,5.5228,9,
534,pasni,PK,25.2631,63.4710,69,
572,la toma,AR,-33.0526,-65.6238,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
   "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arar - nearest hotel: دار أوتيل Dar Otel
bushehr - nearest hotel: هتل رستوران سیراف
south hedland - nearest hotel: Port Haven
cullowhee - nearest hotel: University Inn
tamanrasset - nearest hotel: فندق الأمان
pasni - nearest hotel: Juddi Hotel
la toma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
196,arar,SA,30.9753,41.0381,30,دار أوتيل Dar Otel
295,bushehr,IR,28.9684,50.8385,61,هتل رستوران سیراف
375,south hedland,AU,-20.4089,118.5983,83,Port Haven
418,cullowhee,US,35.3137,-83.1765,52,University Inn
440,tamanrasset,DZ,22.7850,5.5228,9,فندق الأمان
534,pasni,PK,25.2631,63.4710,69,Juddi Hotel
572,la toma,AR,-33.0526,-65.6238,15,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriWorldStreetMap",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)